<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/examples_agents_with_orchestrator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ollama langchain_community --quiet

host="localhost:11434"
modelid="gpt-4o-mini" # "llama3.2"
model_2_id="deepseek-r1:8b"

get_ipython().system_raw("curl -fsSL https://ollama.com/install.sh | sh")
get_ipython().system_raw("ollama serve &")
get_ipython().system_raw(f"ollama pull {modelid}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [2]:
!pip install openai --quiet

In [16]:
modelid= "gpt-4o-mini"

In [17]:
get_ipython().system_raw(f"ollama pull {modelid}")
get_ipython().system_raw(f"ollama pull {model_2_id}")

In [4]:
from openai import OpenAI


llm = OpenAI(
        base_url=f"http://{host}/v1",
        api_key="ollama",  # required, but unused
    )




In [12]:
def call(system_prompt: str, message: str, model: str = modelid ) -> str:
  completion = llm.chat.completions.create(
      model=modelid,
      messages=[
          {"role": "system", "content": system_prompt},
          {
              "role": "user",
              "content": message,
          },
      ],
  )

  return completion.choices[0].message.content

In [18]:
def orchestrator(userinput: str) -> str:

  response_format = {"action":"", "input":"", "next_action":""}

  prompt = """
                Use the context from memory to plan next steps.
                Context:
                []

                You are an expert intent classifier.
                You need will use the context provided and the user's input to classify the intent select the appropriate agent.
                You will rewrite the input for the agent so that the agent can efficiently execute the task.

                Here are the available agents and their descriptions:

               Time Tool Agent - Provides the current time for a given city's timezone like Asia/Kolkata, America/New_York etc. If no timezone is provided, it returns the local time.
               calendar agent - Checks your calendar or schedules a date in your calendar

                User Input:
                {userinput}

                ###Guidelines###
                - Sometimes you might have to use multiple agent's to solve user's input. You have to do that in a loop.
                - The original userinput could have multiple tasks, you will use the context to understand the previous actions taken and the next steps you should take.
                - Read the context, take your time to understand, see if there were many tasks and if you executed them all
                - If there are no actions to be taken, then make the action "respond_to_user" with your final thoughts combining all previous responses as input.
                - Respond with "respond_to_user" only when there are no agents to select from or there is no next_action
                - You will return the agent name in the form of {response_format}
                - Always return valid JSON like {response_format} and nothing else.
  """
  return call ("You are an expert intent classifier.", prompt.format(userinput=userinput, response_format=response_format))

In [19]:
result = orchestrator ("Please schedule a date in my calendar starting in 1 hour.")
result

NotFoundError: Error code: 404 - {'error': {'message': 'model "gpt-4o-mini" not found, try pulling it first', 'type': 'api_error', 'param': None, 'code': None}}